# IMDB 영화리뷰 감성분석 - Conv1D
- Conv1D + Conv1D + Dense
- Conv1D + LSTM + Dense
- Conv1D + Dense + Dense

In [1]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
num_words = 10000
(X_train, y_train), (_, _) = imdb.load_data(num_words=num_words)

17473536/17464789 [==============================] - 0s 0us/step


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, stratify=y_train, test_size=0.2, random_state=seed
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20000,), (5000,), (20000,), (5000,))

In [5]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((20000, 500), (5000, 500))

- Case 1) Conv1D x 2

In [6]:
model1 = Sequential([ 
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.25),
    Conv1D(64, 7, activation='relu'),
    MaxPooling1D(7),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 500, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 494, 64)           44864     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 70, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 64)           0         
 1D)                                                    

In [7]:
model1.compile('adam', 'binary_crossentropy', ['accuracy'])
model1_path = 'best-imdb-conv1d-conv1d.h5'
mc1 = ModelCheckpoint(model1_path, verbose=1, save_best_only=True)
es1 = EarlyStopping(patience=5)

In [8]:
hist1 = model1.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc1, es1]
)

Epoch 1/30
248/250 [============================>.] - ETA: 0s - loss: 0.5363 - accuracy: 0.6916
Epoch 1: val_loss improved from inf to 0.34546, saving model to best-imdb-conv1d-conv1d.h5
250/250 [==============================] - 15s 10ms/step - loss: 0.5344 - accuracy: 0.6931 - val_loss: 0.3455 - val_accuracy: 0.8535
Epoch 2/30
244/250 [============================>.] - ETA: 0s - loss: 0.2573 - accuracy: 0.8973
Epoch 2: val_loss improved from 0.34546 to 0.32187, saving model to best-imdb-conv1d-conv1d.h5
250/250 [==============================] - 2s 9ms/step - loss: 0.2584 - accuracy: 0.8968 - val_loss: 0.3219 - val_accuracy: 0.8620
Epoch 3/30
249/250 [============================>.] - ETA: 0s - loss: 0.1565 - accuracy: 0.9428
Epoch 3: val_loss did not improve from 0.32187
250/250 [==============================] - 2s 9ms/step - loss: 0.1564 - accuracy: 0.9429 - val_loss: 0.3553 - val_accuracy: 0.8597
Epoch 4/30
250/250 [==============================] - ETA: 0s - loss: 0.0814 - accur

In [9]:
best_model1 = load_model(model1_path)
best_model1.evaluate(X_test, y_test)

157/157 [==============================] - 1s 4ms/step - loss: 0.3150 - accuracy: 0.8624


[0.3149791657924652, 0.8623999953269958]

- Case 2) Conv1D + LSTM

In [10]:
model2 = Sequential([ 
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.25),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    LSTM(100),
    Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_1 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                      

In [11]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])
model2_path = 'best-imdb-conv1d-lstm.h5'
mc2 = ModelCheckpoint(model2_path, verbose=1, save_best_only=True)
es2 = EarlyStopping(patience=5)

In [12]:
hist2 = model2.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc2, es2]
)

Epoch 1/30
247/250 [============================>.] - ETA: 0s - loss: 0.4187 - accuracy: 0.7851
Epoch 1: val_loss improved from inf to 0.29747, saving model to best-imdb-conv1d-lstm.h5
250/250 [==============================] - 6s 15ms/step - loss: 0.4171 - accuracy: 0.7864 - val_loss: 0.2975 - val_accuracy: 0.8870
Epoch 2/30
247/250 [============================>.] - ETA: 0s - loss: 0.1989 - accuracy: 0.9228
Epoch 2: val_loss improved from 0.29747 to 0.26544, saving model to best-imdb-conv1d-lstm.h5
250/250 [==============================] - 3s 13ms/step - loss: 0.1998 - accuracy: 0.9223 - val_loss: 0.2654 - val_accuracy: 0.8905
Epoch 3/30
248/250 [============================>.] - ETA: 0s - loss: 0.1188 - accuracy: 0.9586
Epoch 3: val_loss did not improve from 0.26544
250/250 [==============================] - 3s 13ms/step - loss: 0.1188 - accuracy: 0.9584 - val_loss: 0.3053 - val_accuracy: 0.8857
Epoch 4/30
246/250 [============================>.] - ETA: 0s - loss: 0.0718 - accuracy

In [13]:
best_model2 = load_model(model2_path)
best_model2.evaluate(X_test, y_test)

157/157 [==============================] - 1s 5ms/step - loss: 0.2801 - accuracy: 0.8890


[0.28011444211006165, 0.8889999985694885]

- Case 3) Conv1D + Dense

In [14]:
model3 = Sequential([ 
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.25),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_2 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 100)              

In [15]:
model3.compile('adam', 'binary_crossentropy', ['accuracy'])
model3_path = 'best-imdb-conv1d-dense.h5'
mc3 = ModelCheckpoint(model3_path, verbose=1, save_best_only=True)
es3 = EarlyStopping(patience=5)

In [16]:
hist3 = model3.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc3, es3]
)

Epoch 1/30
248/250 [============================>.] - ETA: 0s - loss: 0.5295 - accuracy: 0.7164
Epoch 1: val_loss improved from inf to 0.33980, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 3s 9ms/step - loss: 0.5280 - accuracy: 0.7172 - val_loss: 0.3398 - val_accuracy: 0.8562
Epoch 2/30
246/250 [============================>.] - ETA: 0s - loss: 0.2539 - accuracy: 0.8976
Epoch 2: val_loss improved from 0.33980 to 0.28536, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 2s 9ms/step - loss: 0.2547 - accuracy: 0.8969 - val_loss: 0.2854 - val_accuracy: 0.8860
Epoch 3/30
246/250 [============================>.] - ETA: 0s - loss: 0.1192 - accuracy: 0.9599
Epoch 3: val_loss did not improve from 0.28536
250/250 [==============================] - 2s 8ms/step - loss: 0.1197 - accuracy: 0.9594 - val_loss: 0.3043 - val_accuracy: 0.8802
Epoch 4/30
249/250 [============================>.] - ETA: 0s - loss: 0.0467 - accuracy:

In [17]:
best_model3 = load_model(model3_path)
best_model3.evaluate(X_test, y_test)

157/157 [==============================] - 1s 3ms/step - loss: 0.2841 - accuracy: 0.8798


[0.28412964940071106, 0.879800021648407]